## Import The ImageDataGenerator Library

In [33]:
from keras.preprocessing.image import ImageDataGenerator

# Image Data Agumentation

In [34]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)


In [35]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c1dae236c94e444092d79eb7f6061170 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RiSl-WHZSkZpRolx2hmwp9o2osm6RO3RRTZqnnMD6h6Z',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_3 = client_c1dae236c94e444092d79eb7f6061170.get_object(Bucket='imageclassification-donotdelete-pr-7wigdxumyxq2lq', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [36]:
from io import BytesIO
import zipfile

unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [37]:
pwd

'/home/wsuser/work'

# Loading our data and performing data agumentation

In [38]:
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Rock_Classification_Dataset/train_set',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='categorical')

x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/Rock_Classification_Dataset/test_set',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='categorical')

Found 700 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


# Importing Necessary Libraries

In [39]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# Initializing the model

In [40]:
# Example for initialization
# model=Sequential()

# Adding CNN Layers

In [41]:
classifier = Sequential()

classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(32,(3,3),activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())

In [42]:
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=5,activation='softmax'))

classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

# Compiling the model

In [43]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Fitting the model

In [44]:
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train),
    epochs=20,validation_data=x_test,validation_steps=len(x_test)
)

Epoch 1/20


/tmp/wsuser/ipykernel_164/556800684.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


140/140 [==============================] - 4s 27ms/step - loss: 1.0358 - accuracy: 0.5357 - val_loss: 0.5532 - val_accuracy: 0.8267
Epoch 2/20
140/140 [==============================] - 4s 25ms/step - loss: 0.5961 - accuracy: 0.7329 - val_loss: 0.6552 - val_accuracy: 0.7867
Epoch 3/20
140/140 [==============================] - 3s 24ms/step - loss: 0.5164 - accuracy: 0.7829 - val_loss: 0.5760 - val_accuracy: 0.7933
Epoch 4/20
140/140 [==============================] - 4s 25ms/step - loss: 0.4491 - accuracy: 0.8200 - val_loss: 0.4611 - val_accuracy: 0.8400
Epoch 5/20
140/140 [==============================] - 4s 25ms/step - loss: 0.3936 - accuracy: 0.8429 - val_loss: 0.5437 - val_accuracy: 0.8133
Epoch 6/20
140/140 [==============================] - 3s 25ms/step - loss: 0.4609 - accuracy: 0.8143 - val_loss: 0.3494 - val_accuracy: 0.9200
Epoch 7/20
140/140 [==============================] - 4s 26ms/step - loss: 0.3599 - accuracy: 0.8500 - val_loss: 0.2812 - val_accuracy: 0.9167
Epoch 8/20

In [45]:
!pip install watson-machine-learning-client --upgrade

In [46]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"jiQoqlAvy_zNKMR_ZBJbBhc0gLxIxFgID84S2buUph9H"
}
client=APIClient(wml_credentials)

In [47]:
client=APIClient(wml_credentials)

In [48]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [49]:
space_uid = guid_from_space_name(client,'imageclassification')
print("Space UID="+space_uid)

Space UID=90beb310-5929-4e7c-84aa-22e5719974c9


In [50]:
client.set.default_space(space_uid)

'SUCCESS'

In [51]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [52]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

# Save the Model

In [53]:
classifier.save('rock.h5')

model_json=classifier.to_json()

with open("model-bw.json","w") as json_file:
    json_file.write(model_json)

In [54]:
!tar -zcvf image-classification-model_new.tarz rock.h5

rock.h5


In [55]:
ls -l

total 26392
-rw-rw---- 1 wsuser wscommon 8682843 Sep  3 00:03 image-classification-model_new.tarz
-rw-rw---- 1 wsuser wscommon    2835 Sep  3 00:03 model-bw.json
-rw-rw---- 1 wsuser wscommon 8517121 Sep  2 23:53 my_model.tar.gz
drwxrwx--- 5 wsuser wscommon    4096 Sep  2 23:43 Rock_Classification_Dataset/
-rw-rw---- 1 wsuser wscommon 9810760 Sep  3 00:03 rock.h5


In [56]:
model_details=client.repository.store_model(model='image-classification-model_new.tarz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id=client.repository.get_model_id(model_details)

In [57]:
model_id

'c6b96edc-bc74-4bbe-86bc-741a76a0b446'

In [59]:
client.repository.download(model_id,'my_model1.tar.gz')

Successfully saved model content to file: 'my_model1.tar.gz'


'/home/wsuser/work/my_model1.tar.gz'

# Test the model

In [60]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model=load_model("rock.h5")

img=tensorflow.keras.utils.load_img(r"/home/wsuser/work/Rock_Classification_Dataset/test_set/marble/marble157.jpg",grayscale=False,target_size=(64,64))

x=tensorflow.keras.utils.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=model.predict(x)
pred

array([[0., 0., 1., 0., 0.]], dtype=float32)

# predicting the output

In [61]:
index=['blue calcite','limestone','marble','olivine','red crystal']
result=str(index[int(np.where(pred[0]==1.0)[0][0])])
result

'marble'